In [3]:
import tensorflow as tf
import numpy as np
from tensorflow.keras import layers, models
import os

In [ ]:
# Configuration
IMG_HEIGHT = 316
IMG_WIDTH = 384
BATCH_SIZE = 32
VAL_SPLIT = 0.2
SEED = 42

Sections = [ "Gravel", "OffRoad", "ramp" ]

Section_histories = []

for Section in Sections:
    DATA_DIR = "/home/fizzer/images_for_cnn_training/" + Section + '/images' 
    Section_histories.append(create_and_train())

Epoch 1/100
267/267 [==============================] - 57s 212ms/step - loss: 1.7897 - mae: 0.3536 - val_loss: 0.3883 - val_mae: 0.4033 - lr: 5.0000e-04
Epoch 2/100
176/267 [==================>...........] - ETA: 16s - loss: 0.2038 - mae: 0.2930

In [9]:
def parse_labels(filename):
    # Extract filename from path
    filename_only = tf.strings.split(filename, os.path.sep)[-1]
    
    # Regex pattern to capture Lin and Ang values
    pattern = r'.*_Lin_(-?\d+\.\d{2})_Ang_(-?\d+\.\d{2})\.png$'
    
    # Extract values using regex replace and split
    lin_ang_str = tf.strings.regex_replace(filename_only, pattern, r'\1,\2')
    parts = tf.strings.split(lin_ang_str, ',')
    
    # Convert to floats
    lin = tf.strings.to_number(parts[0], tf.float32)
    ang = tf.strings.to_number(parts[1], tf.float32)
    
    return tf.stack([lin, ang])


In [10]:
def create_dataset(data_dir):
    # List all PNG files
    ds = tf.data.Dataset.list_files(os.path.join(data_dir, "*.png"), shuffle=True)
    
    # Get cardinality as tensor
    cardinality = tf.data.experimental.cardinality(ds)
    
    # Calculate split sizes using TensorFlow operations
    val_size = tf.cast(
        tf.cast(cardinality, tf.float32) * VAL_SPLIT,
        tf.int64
    )
    train_size = cardinality - val_size

    # Split dataset
    train_ds = ds.skip(val_size)
    val_ds = ds.take(val_size)

    # Rest of the processing remains the same...
    def process_path(file_path):
        img = tf.io.read_file(file_path)
        img = tf.io.decode_png(img, channels=3)
        img = tf.image.convert_image_dtype(img, tf.float32)
        return img, parse_labels(file_path)

    train_ds = train_ds.shuffle(10000).map(process_path, num_parallel_calls=tf.data.AUTOTUNE)
    train_ds = train_ds.batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)
    
    val_ds = val_ds.map(process_path, num_parallel_calls=tf.data.AUTOTUNE)
    val_ds = val_ds.batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)
    
    return train_ds, val_ds

In [11]:
def create_model():
    model = models.Sequential([
        layers.Rescaling(1./255, input_shape=(IMG_HEIGHT, IMG_WIDTH, 3)),
        
        # Enhanced convolutional base
        layers.Conv2D(64, (5, 5), activation='relu', padding='same'),
        layers.BatchNormalization(),
        layers.MaxPooling2D((2, 2)),
        layers.Dropout(0.2),
        
        layers.Conv2D(128, (3, 3), activation='relu', padding='same'),
        layers.BatchNormalization(),
        layers.MaxPooling2D((2, 2)),
        layers.Dropout(0.3),
        
        layers.Conv2D(256, (3, 3), activation='relu', padding='same'),
        layers.BatchNormalization(),
        layers.GlobalAveragePooling2D(),
        layers.Dropout(0.4),
        
        # Dense layers with L2 regularization
        layers.Dense(512, activation='relu', kernel_regularizer='l2'),
        layers.Dense(256, activation='relu', kernel_regularizer='l2'),
        layers.Dense(2, activation='tanh')
    ])
    
    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=0.0005),  # Lower initial LR
        loss='mse',
        metrics=['mae']
    )
    return model


In [15]:
def create_and_train():

    train_dataset, val_dataset = create_dataset(DATA_DIR)
    model = create_model()
    
    callbacks = [
        tf.keras.callbacks.EarlyStopping(
            monitor='val_loss',
            patience=50,  # Increased from 10
            min_delta=0.00001,  # Minimum change to qualify as improvement
            mode='min',
            restore_best_weights=True
        ),
        tf.keras.callbacks.ReduceLROnPlateau(
            monitor='val_loss',
            factor=0.5,
            patience=7,  # Wait longer before reducing LR
            verbose=1
        ),
        tf.keras.callbacks.ModelCheckpoint(
            '/home/fizzer/images_for_cnn_training/'+Section+'/best_model.h5',
            save_best_only=True,
            save_weights_only=False,
            monitor='val_loss'
        )
    ]
    
    history = model.fit(
        train_dataset,
        validation_data=val_dataset,
        epochs=100,
        callbacks=callbacks,
        verbose=1
    )
    return history

In [13]:
import matplotlib.pyplot as plt

def plot_training_history(history):
    # Create subplots
    fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(10, 15))
    
    # Plot Loss
    ax1.plot(history.history['loss'], label='Train Loss')
    ax1.plot(history.history['val_loss'], label='Validation Loss')
    ax1.set_title('Model Loss')
    ax1.set_ylabel('MSE')
    ax1.set_xlabel('Epoch')
    ax1.legend()
    
    # Plot MAE
    ax2.plot(history.history['mae'], label='Train MAE')
    ax2.plot(history.history['val_mae'], label='Validation MAE')
    ax2.set_title('Mean Absolute Error')
    ax2.set_ylabel('MAE')
    ax2.set_xlabel('Epoch')
    ax2.legend()
    
    plt.tight_layout()
    plt.show()
    # To save: plt.savefig('training_history.png', dpi=300)

# Visualize after training
plot_training_history(history)

NameError: name 'history' is not defined